# Tinglysningsdata
Script som beriger matrikeltabel med oplysninger fra tingbogen ud fra matrikeloplysninger som slås op i tingbogens [system til system adgang](http://www.tinglysningsretten.dk/etl/Pages/default.aspx) som kan tilgås via [API'et](http://www.tinglysningsretten.dk/etl/hoveddokumenter/Documents/HTTP_API_beskrivelse%20v1.1.docx). Scriptet er opbygget i tre faser:
1. [Input af matrikeldata fra database](#Input)
2. [Berigelse af matrikeldata med data fra tingbogen](#Berigelse)
3. [Output berigede matrikeldata til database](#Output)

In [61]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

#### Databaseforbindelsen opsættes

In [63]:
user = 'xxx'
pw = 'xxx'
port = 5432
host = 'postgres'
db = 'ballerup'
schema = 'proj_tinglysning'
table = 'ballerup'

In [64]:
con = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, pw, host, port, db))

# Input
Laver DF med matrikeloplysninger for Ballerup Kommune. Her er listet to muligheder: 1. som henter data fra csv og 2. som henter fra PostgreSQL

### 1. CSV

In [53]:
# Laver DF med matrikeloplysninger for Ballerup Kommune 
# Fjern .head(5) når det hele skal ind 
matrikel = pd.read_csv('data/matrikel.csv').head()

In [54]:
matrikel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
elavskode    5 non-null int64
matrnr       5 non-null object
esr_ejdnr    5 non-null int64
dtypes: int64(2), object(1)
memory usage: 200.0+ bytes


### 2. PostgreSQL

In [83]:
# Laver DF med matrikeloplysninger for Ballerup Kommune 
# Fjern .head(5) når det hele skal ind 
matrikel = pd.read_sql_query('select elavskode, matrnr, esr_ejdnr from "_00_grundkort"."_00_02_jordstykke"',con=con).head()

In [93]:
# Sætter datatype for ejelav og ejdnr til heltal
matrikel.elavskode = pd.to_numeric(matrikel.elavskode, errors='coerce').astype(np.int64)
matrikel.esr_ejdnr = pd.to_numeric(matrikel.esr_ejdnr, errors='coerce').astype(np.int64)

In [96]:
matrikel

,elavskode,matrnr,esr_ejdnr,tinglysning
0,20151,11bø,1510034160,https://www.tinglysning.dk/tinglysning/unsecre...
1,20151,11cz,1510034152,https://www.tinglysning.dk/tinglysning/unsecre...
2,20151,11cx,1510034101,https://www.tinglysning.dk/tinglysning/unsecre...
3,20151,11cæ,1510034144,https://www.tinglysning.dk/tinglysning/unsecre...
4,20151,11cp,1510017479,https://www.tinglysning.dk/tinglysning/unsecre...


# Berigelse
I denne sektion beriges matriklerne med data fra Tinglysning.

In [86]:
#import requests
import time # Bruges til ikke at overbelaste API'et
import OpenSSL.crypto
import json
import xml.etree.ElementTree as etree

Tænker at det er i nedenstående funktion vi har brug for hjælp til at få hul igennem til API'et. Hvis der er flere tinglysninger tilknyttet en matrikel tænker jeg at ```get_tinlysning``` kan returnere en liste med json-objekter fra kaldene.
<br><br>
[Beskrivelse af hvordan parameteren row fungerer i pandas](https://stackoverflow.com/a/30389492)


In [ ]:
def pfx_to_pem(pfx_path, pfx_password):
    ''' Udpakning af p12 certifikat til PEM  '''
    with tempfile.NamedTemporaryFile(suffix='.pem', delete=False) as t_pem:
        f_pem = open(t_pem.name, 'wb')
        pfx = open(pfx_path, 'rb').read()
        p12 = OpenSSL.crypto.load_pkcs12(pfx, pfx_password)
        f_pem.write(OpenSSL.crypto.dump_privatekey(OpenSSL.crypto.FILETYPE_PEM, p12.get_privatekey()))
        f_pem.write(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, p12.get_certificate()))
        ca = p12.get_ca_certificates()
        if ca is not None:
            for cert in ca:
                f_pem.write(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, cert))
        f_pem.close()
        yield t_pem.name


In [ ]:
def tingbogsparser(tingxml):
    #Fra tinglysningen kommer der xml fil retur med servitutter mm.

    for elem in tingxml.findall('{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}EjendomSummarisk'):

        for child in elem.getchildren():
            for step_child in child.getchildren():
                if step_child.tag == '{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}ServitutSummarisk':

                    for servitutxml in step_child.getchildren():
                        #print(servitutxml.tag, servitutxml.attrib, servitutxml.text)
                        #print('juhuu')
                        if servitutxml.tag =='{http://rep.oio.dk/tinglysning.dk/schema/model/1/}DokumentRevisionIdentifikator':
                            dokid = servitutxml[0].text
                        if servitutxml.tag =='{http://rep.oio.dk/tinglysning.dk/schema/model/1/}TinglysningsDato':
                            tingdato = servitutxml.text
                        if servitutxml.tag =='{http://rep.oio.dk/tinglysning.dk/schema/model/1/}SenestPaategnetDato':
                            paategndato = servitutxml.text
                        if servitutxml.tag =='{http://rep.oio.dk/tinglysning.dk/schema/model/1/}Servitutrettighed':
                            rettighedid = servitutxml[0].text
                        if servitutxml.tag =='{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}OgsaaLystPaaSamling':
                            ogsaalystpaa = servitutxml[0].text
                        if servitutxml.tag =='{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}DokumentAlias':
                            historiskid = servitutxml[0].text
                        if servitutxml.tag =='{http://rep.oio.dk/tinglysning.dk/schema/model/1/}ServitutType':
                            servituttype = servitutxml.text
                        if servitutxml.tag =='{http://rep.oio.dk/tinglysning.dk/schema/elektroniskakt/1/}TillaegstekstSamling':
                            servituttekst = ''.join(servitutxml.itertext())

In [98]:
def get_tinglysning(row):
    """
    Henter data fra tinglysning for hver række i dataframe på baggrund af
    matrikeloplysninger. Bruges i sammenhæng med apply metoden på DF
    """
    ejerlav = str(round(row['elavskode']))
    matnr = row['matrnr']
    #ejedomsnr = row['esr_ejdnr'] #Bruges ikke endnu, men kan bruges som alternativ til matrikelnummer/ejerlav
    # Det anbefales at bruge matrikelnummer/ejerlav.
    url = 'https://www.tinglysning.dk/tinglysning/ssl/ejendom/landsejerlavmatrikel?landsejerlavid={}&matrikelnr={}'.format(ejerlav, matnr)
    with pfx_to_pem('sti_til_p12_certifikat', 'p12_password') as cert:
        r = requests.get(url, cert=cert)

        if r.status_code!= 200:
            print('http error')

        fejl = 0
        try:
            data = json.loads(r.text)
        except ValueError:
            print('Decode error')
            fejl = 1
        #I JSON der returnerers fra tinglysningen er der et uuid for hver ejendom på matriklen. 
        for c in data['items']:
            print (c['uuid'])
            r = requests.get(
            'https://www.tinglysning.dk/tinglysning/ssl/ejdsummarisk/'+c['uuid'],
            cert=cert)
            tingxml = etree.fromstring(r.content)

            tingbogsparser(tingxml)
    
    # Laver et kald hvert 5 sekund
   
    time.sleep(5)
    return url


In [88]:
matrikel['tinglysning'] = matrikel.apply(get_tinglysning, axis=1)

In [89]:
print(matrikel['tinglysning'][1])

https://www.tinglysning.dk/tinglysning/unsecrest/ejendom/landsejerlavmatrikel?landsejerlavid=20151&matrikelnr=11cz


In [90]:
matrikel

,elavskode,matrnr,esr_ejdnr,tinglysning
0,20151,11bø,1510034160,https://www.tinglysning.dk/tinglysning/unsecre...
1,20151,11cz,1510034152,https://www.tinglysning.dk/tinglysning/unsecre...
2,20151,11cx,1510034101,https://www.tinglysning.dk/tinglysning/unsecre...
3,20151,11cæ,1510034144,https://www.tinglysning.dk/tinglysning/unsecre...
4,20151,11cp,1510017479,https://www.tinglysning.dk/tinglysning/unsecre...


# Output
Der er lavet to muligheder for output, CSV og PostgreSQL

### 1. CSV

In [97]:
matrikel.to_csv('data/output.csv')

### 2. PostgreSQL

In [91]:
dtype = {
    'elavskode': sqlalchemy.VARCHAR(), 
    'matrnr': sqlalchemy.VARCHAR(),
    'esr_ejdnr': sqlalchemy.VARCHAR(),
    'tinglysning': sqlalchemy.JSON()
}

In [92]:
matrikel.to_sql(table, con, schema=schema, if_exists='replace', index_label='gid', dtype=dtype)